In [1]:

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [2]:


gpus = tf.config.list_physical_devices('GPU')
print("GPUs available:", gpus)


GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Paths
dataset_dir = "../data/balanced_dataset"   

# Parameters
img_size = 128   # resize images (can try 224 if using transfer learning later)
batch_size = 16
epochs = 10

# Train-validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2   # 80% train, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 14076 images belonging to 47 classes.
Found 3512 images belonging to 47 classes.
Number of classes: 47


In [8]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 batch_normalization_6 (Batc  (None, 126, 126, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_7 (Batc  (None, 61, 61, 64)       256       
 hNormalization)                                                 
                                                      

In [9]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)


Epoch 1/10
114/880 [==>...........................] - ETA: 2:28 - loss: 6.5583 - accuracy: 0.0208

d:\anaconda\ana\envs\tf\lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


880/880 [==============================] - 220s 250ms/step - loss: 4.2178 - accuracy: 0.0271 - val_loss: 3.8452 - val_accuracy: 0.0308
Epoch 2/10
880/880 [==============================] - 228s 259ms/step - loss: 3.8586 - accuracy: 0.0310 - val_loss: 3.8428 - val_accuracy: 0.0310
Epoch 3/10
880/880 [==============================] - 264s 300ms/step - loss: 3.8571 - accuracy: 0.0310 - val_loss: 3.8423 - val_accuracy: 0.0308
Epoch 4/10
880/880 [==============================] - 266s 303ms/step - loss: 3.8427 - accuracy: 0.0298 - val_loss: 3.8419 - val_accuracy: 0.0308
Epoch 5/10
880/880 [==============================] - 260s 295ms/step - loss: 3.8444 - accuracy: 0.0308 - val_loss: 3.8419 - val_accuracy: 0.0308
Epoch 6/10
880/880 [==============================] - 262s 298ms/step - loss: 3.8491 - accuracy: 0.0292 - val_loss: 3.8424 - val_accuracy: 0.0308
Epoch 7/10
880/880 [==============================] - 269s 306ms/step - loss: 3.8468 - accuracy: 0.0292 - val_loss: 3.8420 - val_accura

In [3]:
dataset_dir = "../data/balanced_dataset"  # your dataset
img_size = 224  # EfficientNetB0 expects 224x224
batch_size = 16
epochs = 10


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 14076 images belonging to 47 classes.
Found 3512 images belonging to 47 classes.
Number of classes: 47


In [5]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

# Base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze base layers
base_model.trainable = False

# Add custom classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


16705208/16705208 [==============================] - 3s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)     

In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)


Epoch 1/10
 10/880 [..............................] - ETA: 3:37 - loss: 3.9150 - accuracy: 0.0188

d:\anaconda\ana\envs\tf\lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


138/880 [===>..........................] - ETA: 3:24 - loss: 3.9045 - accuracy: 0.0208

KeyboardInterrupt: 